 # Importar arquivos para leitura

In [1]:
import pandas as pd
import os

In [2]:
base_sip_oem = pd.read_excel(r"C:\Users\helym\projetos_python\jupyter\OEM\content\base_sip_Concluido.xlsx")
#df_intelbras = pd.read_excel(r"E:\ACRÉSCIMOS PREVENTIVA JUNHO 3ª REMESSA.xlsx")

In [3]:
base_sip_oem = base_sip_oem[['IDSIGFI','NUMOS','NOMECLIENTE','CPF','CODIGOENERGISA','UC','TIPODESIGFI','OBRA','ROTA','EXECUTOR','LATLONCON','TIPOCAUSA','DTCONCLUSAO','ENDERECO','MUNICIPIO']]

In [4]:
print(base_sip_oem.shape)
#print(df_intelbras.shape)

(71, 15)


# Realizar leitura de arquivos  .pdf  dentro das pastas correspondente

In [5]:
diretorio_lote = r"C:\Users\helym\OneDrive - ONE ENGENHARIA INDUSTRIA E COMERCIO LTDA\OeM\OeM\00-INTELBRAS\000_Relatórios de Campo\001 RELATORIOS ENVIADOS\AC\0_0-PREVENTIVA\1ªTRACHE\2024\LOTE_17\3ªREMESSA"
#"C:\Users\OneEngInst\OneDrive - ONE ENGENHARIA INDUSTRIA E COMERCIO LTDA\OeM\OeM\00-INTELBRAS\000_Relatórios de Campo\001 RELATORIOS ENVIADOS\0.0-PREVENTIVA\1ªTRACHE\2024\LOTE_16\3ªREMESSA"

In [6]:
# listando os arquivos dentro da pasta.
arquivos_pdf = []
for raiz, diretorios, arquivos in os.walk(diretorio_lote):
    for arquivo in arquivos:
        if arquivo.endswith('.pdf'):
            arquivos_pdf.append(os.path.join(raiz, arquivo))

lista_pdf = pd.DataFrame(arquivos_pdf, columns=['NUMOS'])
lista_pdf['NUMOS'] = lista_pdf['NUMOS'].apply(lambda x: x[-22:-10]).astype('int64')
contagem_arquivos = lista_pdf['NUMOS'].unique().shape[0]
dataset = lista_pdf.merge(base_sip_oem, left_on='NUMOS', right_on='NUMOS', how='left')

In [7]:
dataset[['LATLONCON']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   LATLONCON  0 non-null      object
dtypes: object(1)
memory usage: 364.0+ bytes


In [8]:
# Rastrear duplicidade e soma-las.
dataset['NUMOS'].duplicated().sum()

0

In [9]:
dataset['NUMOS'].count()

29

In [10]:
def encontrar_duplicados(dataset):
    duplicados = dataset[dataset.duplicated(subset='NUMOS')]
    return duplicados['NUMOS']

# Exemplo de uso
duplicados = encontrar_duplicados(dataset)
print(duplicados)

Series([], Name: NUMOS, dtype: int64)


# Realizar a mesclagem com número de OS entre os dados da base do sistema com a lista de arquivos encontrados na pasta

In [11]:
#dataset[['LATITUDE', 'LONGITUDE','ERRO_1','ERRO_2']] = dataset['LATLONCON'].apply(lambda x: pd.Series(x.split(',')))#,'ERRO_1','ERRO_2'
def split_latlon(x):
    if isinstance(x, str):
        return pd.Series(x.split(','))
    else:
        #Se não for uma string, retorna NaN ou valores padrão, dependendo do que você deseja
        return pd.Series([pd.NaT, pd.NaT])

# Aplica a função split_latlon à coluna 'LATLONCON'
dataset[['LATITUDE', 'LONGITUDE']] = dataset['LATLONCON'].apply(split_latlon)#,'erro_1','erro_2'

# Converta as colunas 'LATITUDE' e 'LONGITUDE' para números, se necessário
#dataset[['LATITUDE', 'LONGITUDE']] = dataset[['LATITUDE', 'LONGITUDE']].apply(pd.to_numeric, errors='coerce')

dataset= dataset[['IDSIGFI','UC','OBRA','NOMECLIENTE','CPF','ENDERECO','MUNICIPIO','DTCONCLUSAO','TIPOCAUSA','ROTA','EXECUTOR','TIPODESIGFI','LATITUDE','LONGITUDE','NUMOS']]#,'NUMOS'
lote = dataset.rename(columns={'IDSIGFI':'CÓDIGO ONE','NOMECLIENTE':'NOME DO CLIENTE','CODIGOENERGISA': 'CÓDIGO','UC': 'NÚM.DA UC',
                               'OBRA':'NÚM.DA OBRA','EXECUTOR':'EQUIPE EXECUTORA',
                              'TIPODESIGFI': 'TIPO/SIGFI','TIPOCAUSA':'PREVENTIVA','DTCONCLUSAO':'DATA ATENDIMENTO'})

replacements1 = {'ONEOM001':'EQ.1','ONEOM002':'EQ.2','ONEOM003':'EQ.3','ONEOM004':'EQ.4','ONEOM005':'EQ.5','ONEOM006':'EQ.6','ONEOM007':'EQ.7',
                 'ONEOM008':'EQ.8','ONEOM009':'EQ.9','ONEOM010':'EQ.10','ONEOM011':'EQ.11','ONEOM012':'EQ.12','ONEOM013':'EQ.13','ONEOM014':'EQ.14',
                 'ONEOM015':'EQ.15'}
lote['EQUIPE EXECUTORA'] = lote['EQUIPE EXECUTORA'].replace(replacements1)

replacements2 = {'MANUT PREVENTIVA 1A':'1ªPREVENTIVA','MANUT PREVENTIVA 2A':'2ªPREVENTIVA','MANUT PREVENTIVA 3A':'3ªPREVENTIVA',
                 'MANUT PREVENTIVA 4A':'4ªPREVENTIVA','MANUT PREVENTIVA 5A':'5ªPREVENTIVA'}
lote['PREVENTIVA'] = lote['PREVENTIVA'].replace(replacements2)

replacements3 = {'CRUZEIRO DO SUL':'CRUZEIRO DO SUL','FEIJO':'CRUZEIRO DO SUL','JORDAO':'CRUZEIRO DO SUL',
                 'JURUA':'CRUZEIRO DO SUL','MARECHAL THAUMATURGO':'CRUZEIRO DO SUL','MANCIO LIMA':'CRUZEIRO DO SUL',
                'PORTO WALTER':'CRUZEIRO DO SUL','RODRIGUES ALVES':'CRUZEIRO DO SUL','TARAUACA':'CRUZEIRO DO SUL',
                'ASSIS BRASIL':'BRASILÉIA','SENA MADUREIRA':'RIO BRANCO','RIO AZUL':'CRUZEIRO DO SUL','xxxxxxxxx':'xxxxxxxxxx'}

lote['REGIONAL'] = lote['MUNICIPIO'].replace(replacements3)

lote = lote[['CÓDIGO ONE', 'NÚM.DA UC', 'NÚM.DA OBRA', 'NOME DO CLIENTE', 'CPF',
       'ENDERECO', 'MUNICIPIO','REGIONAL', 'DATA ATENDIMENTO', 'PREVENTIVA', 'ROTA',
       'EQUIPE EXECUTORA', 'TIPO/SIGFI', 'LATITUDE', 'LONGITUDE', 'NUMOS']]

C:\Users\helym\AppData\Local\Temp\ipykernel_15252\2816517730.py:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lote['EQUIPE EXECUTORA'] = lote['EQUIPE EXECUTORA'].replace(replacements1)
C:\Users\helym\AppData\Local\Temp\ipykernel_15252\2816517730.py:27: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lote['PREVENTIVA'] = lote['PREVENTIVA'].replace(replacements2)
C:\Users\helym\AppData\Local\Temp\ipykernel_15252\2816517730.py:34: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version

In [12]:
lote

,CÓDIGO ONE,NÚM.DA UC,NÚM.DA OBRA,NOME DO CLIENTE,CPF,ENDERECO,MUNICIPIO,REGIONAL,DATA ATENDIMENTO,PREVENTIVA,ROTA,EQUIPE EXECUTORA,TIPO/SIGFI,LATITUDE,LONGITUDE,NUMOS
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT,NaT,202400032142
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT,NaT,202400030765
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT,NaT,202400030781
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT,NaT,202400030757
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT,NaT,202400030898
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT,NaT,202400030773
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT,NaT,202400030872
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT,NaT,202400020220
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT,NaT,202400020162
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT,NaT,202400018562


In [13]:
# Salvando a planilha( .xlsx)
dataframe_excel = pd.ExcelWriter(r"C:\Users\helym\projetos_python\jupyter\OEM\output\1ºtrache_intelbras_preventiva_lote17_2ªremessa_2024.xlsx")
lote.to_excel(dataframe_excel, index=False)
dataframe_excel.close()